We count principal $(\ell_1^{e_1}, ..., \ell_r^{e_r})$-isogeny cycles subject to these criteria:
1. Each cycle has a designated starting vertex, just as a cycle would in the graph theoretic sense.
2. We count cycles up to the equivalence of the canonical decomposition of the endomorphism they generate. This means if two different graph theoretic cycles generate the same endomorphism (up to sign), we only count one cycle.

Different counts come with different restrictions on the parameters $p$, $L=\{\ell_1, ..., \ell_r\}$ and exponents $e_i$. The algorithms can be very computationally heavy for larger parameters, as they essentially all require computing the full $L$-isogeny graph to begin.
By default we use the parameters for the example in the paper, $p=61$, $L = \{2,3\}$ with exponents $e_1=e_2=1$.

We use the following different methods:
1. Using $\ell_i$-powers of Brandt matrices. Works for $p \equiv 1 \mod 12$.
2. Count using approximate traces of Brandt matrices. Works for $p \equiv 1 \mod 12$.
3. Count using ideal relations of quadratic orders. Works for $p \equiv 1 \mod 12$, with exponents $e_1 = ... = e_r = 1$, and $\prod \ell_i < \frac{p}{4}$.
4. Brute force count by finding cycles in isogeny graph. Works for all parameter choices.

In [ ]:
load("utils.sage")

p = 61
L = [2,3]
exponents = [1, 1]
G = get_L_graph(p, L)

### 1. Using $\ell_i$-powers of Brandt matrices

Requires $p \equiv 1 \mod 12$.

In [22]:
if mod(p, 12) != 1:
    raise Exception("Only p 1 modulo 12 implemented")

n = (p-1)//12
M = product([L[r]^exponents[r] for r in range(0, len(L))]) # the degree of the endomorphism, ell_1^e_1 ... ell_r^e_r
S = BrandtModule(p)
Id = matrix.identity(n)
H = S.hecke_matrix(M) # the matrix B(ell_1^e_1 ... ell_r^e_r)
# Find the indices i where e_i >= 2
ell_es_BT = []
for r in range(0, len(L)):
    if exponents[r] >=2:
        ell_es_BT.append(r)
R = len(ell_es_BT)
# Sum over all Brandt matrices B(ell_1^e_1 ... ell_r^e_r) were we remove 2 from different subsets of exponents
for k in range(1,R+1):
    ssets = Subsets(ell_es_BT,k,submultiset=True)
    for e_set in ssets:
        M_new = M
        for r in e_set:
            M_new = M_new/(L[r]^2)
        if M_new == 1:
            H = H + Id*(-1)^k
        else:
            H = H + S.hecke_matrix(M_new)*(-1)^k

# Take the trace of the resulting sum of matrices
H.trace()

10

### 2. Count using approximate traces of Brandt matrices

Requires $p \equiv 1 \mod 12$.

This count is significantly faster, but less precise.

In [24]:
def approximate_trace_brandt_matrix(L, exponents):
    """Returns approximation of the trace of the Brandt matrix B(ell_1^e_1 ... ell_r^e_r)  with ell_i and e_i given in lists L and exponents."""
    def d_enumerate(L, exponents):
        yield 1
        exps = [0 for r in range(0, len(L))]
        while True:
            exps[0] = exps[0] + 1
            for r in range(0, len(L)):
                if exps[r] == exponents[r] + 1:
                    if r+1 == len(L):
                        return
                    exps[r] = 0
                    exps[r+1] = exps[r+1] + 1
            yield product([L[i]^(exps[i]) for i in range(0, len(L))])

    prod = 1
    for i in range(0, len(L)):
        ell = L[i]
        ei = exponents[i]
        term = (ell^(ei + 1) - 1) / (ell - 1)
        prod = prod * term

    summand = 0
    degree = product([L[i]^(exponents[i]) for i in range(0, len(L))])
    for d in d_enumerate(L, exponents):
        summand += min(d, degree/d)

    return prod - (1/2)*summand

# We do the same calculation as the previous counting method, replacing the matrix sum as a sum of traces, and using the above function to approximate them

if mod(p, 12) != 1:
    raise Exception("Only p 1 modulo 12 implemented")

n = (p-1)//12
ell_es_BT = []
for r in range(0, len(L)):
    if exponents[r] >=2:
        ell_es_BT.append(r)
R = len(ell_es_BT)
H_trace = approximate_trace_brandt_matrix(L, exponents)
summand = H_trace
for k in range(1,R+1):
    ssets = Subsets(ell_es_BT,k,submultiset=True)
    for e_set in ssets:
        new_exponents = copy(exponents)
        for r in e_set:
            new_exponents[r] = new_exponents[r] - 2
        if sum(new_exponents) == 0:
            summand += ((-1)^k)*n
        else:
            summand += ((-1)^k)*approximate_trace_brandt_matrix(L, new_exponents)
summand

9

### 3. Count using ideal relations of quadratic orders

Requires $p \equiv 1 \mod 12$, with exponents $e_1 = ... = e_r = 1$, and $\prod \ell_i < \frac{p}{4}$.

In [5]:
def get_cyclic_ideal_products(exponents_r, prime_ideals_above_ell, O_quad):
    """Recursively finds all cyclic ideal products of given list of prime ideals to given list of exponents."""
    for I in prime_ideals_above_ell[0]:
        if len(exponents_r) > 1:
            for I2 in get_cyclic_ideal_products(exponents_r[1:], prime_ideals_above_ell[1:], O_quad):
                yield (I**exponents_r[0]) * I2
        else:
            yield I**exponents_r[0]

assert(mod(p, 12) == 1)
assert(False not in [ei == 1 or ei == 0 for ei in exponents])

# restrict to primes ell and exponents, where exponents are non-zero in cycle degree
L_r = []
exponents_r = []
for i in range(0, len(L)):
    if exponents[i] > 0:
        L_r.append(L[i])
        exponents_r.append(exponents[i])

degree = product([L_r[i]^(exponents_r[i]) for i in range(0, len(L_r))])
assert(degree <= p/4)

# For imaginary quadratic order need trace^2 <= 4*degree
summand = 0
for trace in range(0, floor(sqrt(4*degree)) + 1):
    disc = trace**2 - 4*degree
    if disc == 0: continue
    disc_squarefree_part = disc.squarefree_part()
    disc_square_part = sqrt(disc / disc_squarefree_part)
    K = QuadraticField(disc_squarefree_part)
    z = K.gens()[0] # generator of field
    g = (trace + disc_square_part*z)/2 # generator of the quadratic order, with norm 'degree' and trace 'trace'
    O_quad = K.order([1, g])

    print("")
    print("Found norm " + str(degree) + " element " + imaginary_quadratic_elt_to_string(g))

    # Check p is not split
    if len(quadratic_ideals_above_prime(O_quad, p)) == 2:
        print("but p is split in the order so ignore it")
        continue

    prime_ideals_above_ell = [quadratic_ideals_above_prime(O_quad, ell) for ell in L_r]

    # make sure every prime ell has at least one ideal above it
    if [] in prime_ideals_above_ell:
        print("but some ells are inert in the order so ignore it")
        continue

    # make sure when a ramified prime has to appear, the exponent is not more than 1 (otherwise we'd get a non-cyclic product)
    should_break = False
    for i in range(0, len(L_r)):
        if exponents_r[i] > 1 and len(prime_ideals_above_ell[i]) ==  1:
            print("but " + str(L_r[i]) + " is ramified so cannot appear " + str(exponents_r[i]) + " times in the ideal product.")
            should_break = True
            break
    if should_break: break
    
    # construct ideal products of the right norm
    ideal_product = []
    for I in get_cyclic_ideal_products(exponents_r, prime_ideals_above_ell, O_quad):
        # Check ideal product is principal
        if I.is_principal():
            ideal_product.append(I)
    
    # Make sure the ideal products are unique
    ideal_product = list(set(ideal_product))
    print("and within the order it generates, found " + str(len(ideal_product)) + " principal cyclic ideals of correct norm.")

    # At term to sum in the count
    hO = O_quad.class_number()
    summand += 2 * hO * len(ideal_product)

print("")
print("Total cycle count: " + str((product([exp.factorial() for exp in exponents_r]) / sum(exponents_r)) * summand))


Found norm 6 element sqrt(-6)
and within the order it generates, found 1 principal cyclic ideals of correct norm.

Found norm 6 element 1/2 + 1/2.sqrt(-23)
and within the order it generates, found 2 principal cyclic ideals of correct norm.

Found norm 6 element 1 + sqrt(-5)
but p is split in the order so ignore it

Found norm 6 element 3/2 + 1/2.sqrt(-15)
but p is split in the order so ignore it

Found norm 6 element 2 + sqrt(-2)
and within the order it generates, found 2 principal cyclic ideals of correct norm.

Total cycle count: 10


### 4. Brute force count by finding cycles in isogeny graph

Works for all parameter choices.

This algorithm computes a full $\ell$-torsion basis every time we check an isogeny is cyclic. Hence it only performs acceptably if the primes $\ell \in L$ have torsion bases defined over relatively small field extensions OR if primes $\ell_i$ with torsion bases over a larger extension have exponents $e_i \leq 1$.
For a more efficient implementation there are better methods to check if an isogeny is cyclic, and caching of torsion bases could also give a significant speedup.

In [6]:
from sage.schemes.elliptic_curves.hom import compare_via_evaluation

def find_paths_enum(start_vertex, degree):
    """Recursively finds all paths of a given degree from a given starting vertex in the graph"""
    if degree == 1:
        yield []
    else:
        for r in range(0, len(G.edges())):
            edge = G.edges()[r]
            if edge[0] != start_vertex:
                continue
            ell = ZZ(edge[2])
            if mod(degree, ell) != 0:
                continue
            for path in find_paths_enum(edge[1], degree/ell):
                yield [r] + path

def find_cycles_enum(G, start_vertex, L, exponents):
    """Finds all cycles from starting vertex in graph, given prime list L and list of exponents for the degree"""
    degree = product([L[i]^(exponents[i]) for i in range(0, len(L))])
    for path in find_paths_enum(start_vertex, degree):
        last_edge = G.edges()[path[-1]]
        end_vertex = last_edge[1]
        if start_vertex == end_vertex:
            yield path

def get_arbitrary_assigned_isogeny(curves, j_invs, isogeny):
    """Given list of curves representing vertices in graph, and an isogeny from such a curve,
    post-composes with isomorphism to ensure codomain also lies within list of curves.
    This is essentially obtaining the isogeny from the 'arbitrary assignment' defined in the paper."""
    actual_codomain = isogeny.codomain()
    target_codomain = curves[j_invs.index(actual_codomain.j_invariant())]
    iso = actual_codomain.isomorphism_to(target_codomain)
    return isogeny.post_compose(iso)

def is_isogeny_ell_cyclic(isogeny, ell):
    """Returns True if isogeny does not contain entire ell torsion in its kernel."""
    E1 = isogeny.domain()
    E2 = isogeny.codomain()
    F_ext = E1.division_field(ell) # field extension where full ell-torsion is defined
    E1_ext = E1.base_extend(F_ext) # define curves over the field extension
    E2_ext = E2.base_extend(F_ext)
    # compute ell-torsion basis
    tb = E1_ext.torsion_basis(ell)
    # check isogeny takes torsion basis to identity
    for P in tb:
        if isogeny._eval(P) != E2_ext(isogeny.codomain()[0]):
            return True
    return False

def is_isogeny_L_cyclic(isogeny, ells):
    """Returns true if isogeny does not contain any entire ell torsion, for some prime ell in a set ells."""
    for ell in ells:
        if not is_isogeny_ell_cyclic(isogeny, ell):
            return False
    return True


# For checking if isogenies are cyclic later on, we need a list of primes such that ell^2 divides the degree
ell_for_cyclic_test = [L[i] for i in range(0, len(L)) if exponents[i] > 1]

# Get curves representing vertices of the graph
Fp2 = GF(p^2)
j_invs = [Fp2(v) for v in G.vertices()]
curves = [EllipticCurve_from_j(j) for j in j_invs]

# Get isogenies representing edges of the graphs from an arbitrary assignment defined by the curves above
isogeny_edge_list = [None for r in range(0, len(G.edges()))]
for r in range(0, len(curves)):
    E = curves[r]
    for ell in L:
        ell_isogenies = E.isogenies_prime_degree(ell)
        ell_isogenies = [get_arbitrary_assigned_isogeny(curves, j_invs, isogeny) for isogeny in ell_isogenies] # isogenies for a compatible arbitrary assignment
        # we need to specify which edge corresponds to each isogeny, so attempt to fill in array 'isogeny_edge_list'
        #    put isogeny in entry r if G.edges()[r] has correct start/end vertices and degree
        for ell_isogeny in ell_isogenies:
            assigned = False
            for r in range(0, len(G.edges())):
                if isogeny_edge_list[r] != None:
                    continue
                if ZZ(G.edges()[r][2]) != ell:
                    continue
                if Fp2(G.edges()[r][0]) == ell_isogeny.domain().j_invariant() and Fp2(G.edges()[r][1]) == ell_isogeny.codomain().j_invariant():
                    isogeny_edge_list[r] = ell_isogeny
                    assigned = True
                    break
            if not assigned:
                raise Exception("Isogeny could not be assigned to edge")
assert(None not in isogeny_edge_list)


# Iterate over all graph theoretic cycles in the graph
cycles = []
for start_vertex in G.vertices():
    cycle_endomorphisms_for_this_vertex = []
    for cycle in find_cycles_enum(G, start_vertex, L, exponents):
        # a cycle is a list of indices of edges [edge_idx_1, edge_idx_2, ...] such that edge i is given by G.edges()[edge_idx_i],  and isogeny  isogeny_edge_list[i]
        #    as a "closed walk", edges can repeat

        # get endomorphism for cycle by composing the corresponding isogenies
        first_isogeny = isogeny_edge_list[cycle[0]]
        endomorphism = first_isogeny
        for r in range(1, len(cycle)):
            edge_idx = cycle[r]
            next_isogeny = isogeny_edge_list[edge_idx]
            endomorphism = endomorphism.post_compose(next_isogeny)
        
        # check the endomorphism is cyclic, required to be principal / non-backtracking
        if not is_isogeny_L_cyclic(endomorphism, ell_for_cyclic_test):
            continue

        # check we haven't already found a cycle from the same vertex with the same canonical decomposition, i.e. the same endomorphism up to sign
        already_found = False
        for old_endo in cycle_endomorphisms_for_this_vertex:
            # we can try checking equality, but this doesn't always work
            if old_endo == endomorphism:
                already_found = True
                break
            if old_endo == -endomorphism: # check up to sign
                already_found = True
                break
            # check by evaluating sufficiently many points
            if compare_via_evaluation(old_endo, endomorphism):
                found = True
                break
            if compare_via_evaluation(old_endo, -endomorphism):
                found = True
                break
        if already_found:
            continue

        # This is a new cycle, add to lists
        cycles.append(cycle)
        cycle_endomorphisms_for_this_vertex.append(endomorphism)

len(cycles)

10